<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [1]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

In [2]:
df = pd.read_csv('../module1-text-data/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [7]:
import spacy
nlp = spacy.load("en_core_web_lg")

def tokenize(document):
    doc = nlp(document)
    return [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

In [8]:
# get a list of tokens for each review
tokens = df['reviews.text'].apply(tokenize)
len(tokens)

28332

In [9]:
# checking num rows to ensure that token len matches up
df.shape

(28332, 24)

In [10]:
# creating dictionary from all tokens
id2word = corpora.Dictionary(tokens)

In [11]:
# simple search to get id from token
id2word.token2id['battery']

4

In [12]:
import sys
print(sys.getsizeof(id2word))
print(sys.getsizeof(tokens))

56
4661248


In [13]:
# how many words in the dictionary?
len(id2word.keys())

9727

In [14]:
# Let's remove extreme values from the dataset
id2word.filter_extremes(no_below=5, no_above=0.95)

In [15]:
# new len?
len(id2word.keys())

3087

In [17]:
# creating a corpus of bag of word representations from all reviews 
corpus = [id2word.doc2bow(text) for text in list(tokens)]

In [18]:
# create LDA model
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=12
                  )

In [19]:
# get topics from LDA model
lda.print_topics()

[(0,
  '0.082*"tablet" + 0.039*"kid" + 0.037*"love" + 0.036*"great" + 0.025*"game" + 0.025*"old" + 0.020*"year" + 0.018*"good" + 0.013*"buy" + 0.013*"son"'),
 (1,
  '0.042*"kid" + 0.041*"use" + 0.039*"easy" + 0.036*"tablet" + 0.035*"love" + 0.027*"year" + 0.024*"old" + 0.023*"great" + 0.022*"time" + 0.019*"set"'),
 (2,
  '0.043*"good" + 0.039*"tablet" + 0.032*"like" + 0.020*"price" + 0.019*"quality" + 0.017*"Amazon" + 0.011*"pretty" + 0.011*"product" + 0.010*"$" + 0.009*"great"'),
 (3,
  '0.032*"buy" + 0.031*"tablet" + 0.030*"need" + 0.020*"gift" + 0.020*"Christmas" + 0.013*"use" + 0.013*"purchase" + 0.013*"love" + 0.013*"Amazon" + 0.013*"thing"'),
 (4,
  '0.039*"tablet" + 0.027*"Amazon" + 0.025*"app" + 0.018*"store" + 0.014*"screen" + 0.014*"device" + 0.011*"Google" + 0.011*"good" + 0.009*"like" + 0.009*"price"'),
 (5,
  '0.054*"good" + 0.037*"tablet" + 0.031*"screen" + 0.028*"great" + 0.024*"price" + 0.022*"easy" + 0.022*"use" + 0.019*"nice" + 0.018*"size" + 0.012*"picture"'),
 (6,
 

In [20]:
# get words of topics
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [21]:
# create topic list
topics = [' '.join(t[0:5]) for t in words]

In [22]:
# view topic list
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
tablet kid love great game

------ Topic 1 ------
kid use easy tablet love

------ Topic 2 ------
good tablet like price quality

------ Topic 3 ------
buy tablet need gift Christmas

------ Topic 4 ------
tablet Amazon app store screen

------ Topic 5 ------
good tablet screen great price

------ Topic 6 ------
app tablet little kid slow

------ Topic 7 ------
work great buy good tablet

------ Topic 8 ------
charge great love Kindle tablet

------ Topic 9 ------
love buy old year kindle

------ Topic 10 ------
read like use book easy

------ Topic 11 ------
great price battery product work

------ Topic 12 ------
battery price great good work

------ Topic 13 ------
battery long brand good work

------ Topic 14 ------
Fire Kindle easy book tablet



In [23]:
# get topic distrabution of each review 
distro = [lda[d] for d in corpus]

def update(doc):
        d_dist = {k:0 for k in range(0,15)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

In [24]:
# dropping uneeded columns from df
df = df[['name', 'reviews.rating', 'reviews.text']]

In [29]:
# create a df detailing the distribution of each topic for each review
names = [name for name in df['name']]
df_distro = pd.DataFrame.from_records(new_distro, index=names)
df_distro.columns = topics
df_distro['name'] = df_distro.reset_index()['index'].tolist()
df_distro.head()

,tablet kid love great game,kid use easy tablet love,good tablet like price quality,buy tablet need gift Christmas,tablet Amazon app store screen,good tablet screen great price,app tablet little kid slow,work great buy good tablet,charge great love Kindle tablet,love buy old year kindle,read like use book easy,great price battery product work,battery price great good work,battery long brand good work,Fire Kindle easy book tablet,name
AmazonBasics AAA Performance Alkaline Batteries (36 Count),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.196103,0.000000,0.000000,0.000000,0.000000,0.322228,0.414975,0.000000,AmazonBasics AAA Performance Alkaline Batterie...
AmazonBasics AAA Performance Alkaline Batteries (36 Count),0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.430183,0.011112,0.011112,0.425365,0.011112,AmazonBasics AAA Performance Alkaline Batterie...
AmazonBasics AAA Performance Alkaline Batteries (36 Count),0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.367871,0.016667,0.415455,0.016667,AmazonBasics AAA Performance Alkaline Batterie...
AmazonBasics AAA Performance Alkaline Batteries (36 Count),0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.844443,0.011111,AmazonBasics AAA Performance Alkaline Batterie...
AmazonBasics AAA Performance Alkaline Batteries (36 Count),0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.307523,0.011111,0.548031,0.011111,AmazonBasics AAA Performance Alkaline Batterie...


In [32]:
# get the mean topic of each product
df_distro.groupby(by="name").mean()

,tablet kid love great game,kid use easy tablet love,good tablet like price quality,buy tablet need gift Christmas,tablet Amazon app store screen,good tablet screen great price,app tablet little kid slow,work great buy good tablet,charge great love Kindle tablet,love buy old year kindle,read like use book easy,great price battery product work,battery price great good work,battery long brand good work,Fire Kindle easy book tablet
name,,,,,,,,,,,,,,,
"All-New Fire 7 Tablet with Alexa, 7"" Display, 8 GB - Marine Blue",0.137757,0.088382,0.065670,0.037478,0.040482,0.070372,0.024148,0.082988,0.047189,0.127546,0.066066,0.054656,0.023873,0.016778,0.051353
"All-New Fire HD 8 Kids Edition Tablet, 8 HD Display, 32 GB, Blue Kid-Proof Case",0.155275,0.248264,0.031231,0.029051,0.038712,0.035331,0.052197,0.043709,0.023554,0.158090,0.045785,0.025566,0.014057,0.013540,0.024696
"All-New Fire HD 8 Kids Edition Tablet, 8 HD Display, 32 GB, Pink Kid-Proof Case",0.164438,0.245468,0.026633,0.024908,0.037813,0.017858,0.050347,0.049556,0.018814,0.169522,0.039703,0.028453,0.018562,0.016450,0.029317
"All-New Fire HD 8 Tablet with Alexa, 8 HD Display, 16 GB, Marine Blue - with Special Offers",0.064142,0.056690,0.054357,0.050936,0.068373,0.111968,0.034908,0.047985,0.022995,0.120311,0.074414,0.064018,0.023728,0.018764,0.119668
"All-New Fire HD 8 Tablet with Alexa, 8 HD Display, 32 GB, Marine Blue - with Special Offers",0.056881,0.053798,0.054273,0.037374,0.049996,0.090109,0.048503,0.037939,0.038788,0.160468,0.074748,0.039614,0.025979,0.024664,0.149101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Kindle PowerFast International Charging Kit (for accelerated charging in over 200 countries),0.000000,0.096725,0.029540,0.225429,0.000000,0.011121,0.009598,0.068446,0.280867,0.000000,0.000000,0.016742,0.170545,0.014455,0.056120
"Kindle Voyage E-reader, 6 High-Resolution Display (300 ppi) with Adaptive Built-in Light, PagePress Sensors, Free 3G + Wi-Fi - Includes Special Offers",0.014914,0.022254,0.050309,0.017766,0.026945,0.076184,0.021809,0.015847,0.092293,0.122473,0.173390,0.079154,0.017261,0.030909,0.190537
"Kindle Voyage E-reader, 6 High-Resolution Display (300 ppi) with Adaptive Built-in Light, PagePress Sensors, Wi-Fi - Includes Special Offers",0.006079,0.016640,0.062514,0.024610,0.033449,0.118948,0.022690,0.016584,0.214317,0.093276,0.106093,0.056945,0.014463,0.029236,0.131982


## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling